# Answering business questions for a record store

In [1]:
import sqlite3
import pandas as pd

In [2]:
# Prints results of query
def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)
# Commits changes to database    
def run_command(c):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(c)
# Easily check state of database        
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


Finding out which genres sell the best in the USA

In [3]:
genre_sales_usa = """ WITH gt AS(
                        SELECT g.name, t.track_id FROM genre g
                        JOIN track t ON g.genre_id=t.genre_id
                        ),
                        
                     gtil AS (
                        SELECT gt.name, il.quantity, il.invoice_id
                        FROM gt
                        JOIN invoice_line il ON gt.track_id=il.track_id
                        ),
                        
                    gtill AS (
                        SELECT gtil.name, gtil.quantity, i.billing_country 
                        FROM gtil
                        JOIN invoice i ON gtil.invoice_id=i.invoice_id
WHERE i.billing_country = 'USA'),

                    total_count AS (
                    SELECT COUNT(*) FROM gtill) 

SELECT gtill.name, CAST(SUM(gtill.quantity) AS FLOAT) AS genre_sales, 
    CAST(SUM(gtill.quantity) AS FLOAT) / (SELECT * FROM total_count) *100 genre_sales_percent
FROM gtill
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

"""
run_query(genre_sales_usa)

,name,genre_sales,genre_sales_percent
0,Rock,561.0,53.377735
1,Alternative & Punk,130.0,12.369172
2,Metal,124.0,11.798287
3,R&B/Soul,53.0,5.042816
4,Blues,36.0,3.425309
5,Alternative,35.0,3.330162
6,Latin,22.0,2.093245
7,Pop,22.0,2.093245
8,Hip Hop/Rap,20.0,1.902950
9,Jazz,14.0,1.332065


Finding best sales employee

In [4]:
total_sales_of_employee = """ WITH ic AS(
                                SELECT i.total, i.invoice_date, c.support_rep_id
                                FROM invoice i
                                Join customer c ON i.customer_id=c.customer_id
                                ),
                                
                            ice AS(
                            SELECT e.employee_id, e.title, e.hire_date, ic.total, ic.invoice_date
                            FROM ic
                            JOIN employee e ON ic.support_rep_id=e.employee_id
                            )

SELECT employee_id, SUM(total), hire_date, title
FROM ice
WHERE invoice_date >= '2018/01/01' or invoice_date >= '2018-01-01'
GROUP BY 1 ORDER BY 2 DESC;

"""
run_query(total_sales_of_employee)

,employee_id,SUM(total),hire_date,title
0,3,1198.89,2017-04-01 00:00:00,Sales Support Agent
1,4,1167.21,2017-05-03 00:00:00,Sales Support Agent
2,5,1141.47,2017-10-17 00:00:00,Sales Support Agent


I chose to only look at sales two monthes after the last employee was hired on to account for a learning curve. While employee id 3 has the best sales the order in best sales correlates with their hire dates

For each country I will find the total number of customers, total number of sales, the avg revenue per customer, and the average revenue per order. Also, I will group all countries with only one customer under 'Other' when evaluating the stats.

In [5]:
country_statistics_query = '''
WITH country_customer_count 
     AS (SELECT customer.country, 
                COUNT(DISTINCT customer.customer_id) AS country_count 
           FROM customer 
          GROUP BY customer.country
         ), 
        
     country_statistics
     AS (SELECT CASE 
              WHEN ccc.country_count = 1 THEN "Other" 
              ELSE customer.country 
            END                                  AS countries_and_other, 
            COUNT(DISTINCT customer.customer_id) AS number_of_customers, 
            SUM(invoice.total)                   AS total_sales, 
            COUNT(DISTINCT invoice.invoice_id)   AS count_transactions,
            SUM(invoice.total) / 
            COUNT(DISTINCT customer.customer_id) AS avg_sales_per_customer,
            SUM(invoice.total) / 
            COUNT(DISTINCT invoice.invoice_id)   AS avg_sales_per_order,
            CASE 
              WHEN ccc.country_count = 1 THEN 1
              ELSE 0 
            END                                  AS sort_by_other 
       FROM customer 
            INNER JOIN country_customer_count AS ccc 
                    ON ccc.country = customer.country 
            INNER JOIN invoice 
                    ON invoice.customer_id = customer.customer_id
      GROUP BY countries_and_other
      ORDER BY sort_by_other ASC, 
               total_sales DESC
      )
               
SELECT countries_and_other    AS "Country", 
       number_of_customers    AS "Total Customers", 
       total_sales            AS "Total Sales", 
       avg_sales_per_customer AS "Average Revenue Per Customer", 
       avg_sales_per_order    AS "Average Revenue Per Order" 
  FROM country_statistics;
 '''
run_query(country_statistics_query)

,Country,Total Customers,Total Sales,Average Revenue Per Customer,Average Revenue Per Order
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,Czech Republic,2,273.24,136.620000,9.108000
6,United Kingdom,3,245.52,81.840000,8.768571
7,Portugal,2,185.13,92.565000,6.383793
8,India,2,183.15,91.575000,8.721429
9,Other,15,1094.94,72.996000,7.448571


As you can see the Czech Republic has the best average revenue per customer, and the average revenue per order

I will check to see if this company should start buying only top songs from albums instead of whole albums if not that many people are buying full albums, so they could stop paying for songs that they don't sell.

In [13]:
album_or_track_purchase = """ WITH album_3_tracks_or_more AS(
                                SELECT album_id, COUNT(DISTINCT track_id) num_tracks, track_id
                                FROM track
                                GROUP BY album_id

                                ),
                                
                                  purchased_tracks AS(
                                    SELECT il.invoice_id,
                                           il.track_id, 
                                           t.album_id,
                                           COUNT(DISTINCT il.track_id) num_tracks
                                    FROM invoice_line il
                                    JOIN track t ON il.track_id = t.track_id
                                    GROUP BY il.invoice_id),
                                    
                                    
                                  percentage_sales_from_album  AS(
                                      SELECT CAST(COUNT(DISTINCT p.invoice_id) AS FLOAT) / (SELECT COUNT(DISTINCT invoice_id)
                                       FROM invoice_line) AS percentage_album_sales
                                    FROM album_3_tracks_or_more a
                                    JOIN purchased_tracks p ON a.album_id=p.album_id
                                    GROUP BY a.album_id
                                    HAVING a.num_tracks = p.num_tracks
                                    )

SELECT SUM(percentage_album_sales) * 100 percent_album_sales,
       100 - SUM(percentage_album_sales) * 100 percent_track_sales
       FROM percentage_sales_from_album

;
"""
run_query(album_or_track_purchase)

,percent_album_sales,percent_track_sales
0,18.241042,81.758958


Only including albums with four songs or more; album sales contributed to about 10% of total sales. Depending on how much money the business will save it is hard to say wether or not they would benifit from only purchasing top songs from albums instead of whole albums. 